# Image classification with `skorch`

**Make sure you look at [`Intro to image classification`](Intro_to_image_classification.ipynb) before coming here.**

**I recommend using the notebook [`Image classification with PyTorch`](Image_classification_with_Scikit-Learn.ipynb) instead of this one.**

We'll use `Pytorch` together with supporting libraries `tensorlayers` and `skorch` to train a classifier for fossil images.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## The fossil dataset

Let's generate a workflow to classify images using a CNN.
We'll make use of a collection of functions in `utils.py` to help process the images found in the `data/fossils` folder.

In [ ]:
X = np.load('../data/fossils/X.npy')
y = np.load('../data/fossils/y.npy')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
X_train.shape

In [ ]:
plt.imshow(X_train[1].reshape(32,32))
plt.colorbar()

## A convolutional neural network with skorch

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(np.append(y_train, y_val))

y_train = encoder.transform(y_train)
y_val = encoder.transform(y_val)

Skorch can use NumPY arrays, which is nice, but for some reason they have to be single precision (i.e. 32-bit floats).

In [ ]:
X_train = X_train.reshape(-1, 1, 32, 32).astype(np.float32)
X_val = X_val.reshape(-1, 1, 32, 32).astype(np.float32)

In [ ]:
import torch
from torch import nn
import torchlayers as tl

# torch.nn and torchlayers can be mixed easily.
net_arch = torch.nn.Sequential(
    tl.Conv(32, kernel_size=3),  # specify ONLY out_channels
    nn.ELU(),  # use torch.nn wherever you wish
    tl.BatchNorm(),
    tl.Conv(16, kernel_size=3),
    nn.ELU(),  
    tl.BatchNorm(),
    tl.GlobalMaxPool(),
    tl.Linear(100), # Add a fully connected hidden layer
    tl.ELU(), # Activate the hidden layer
    tl.Linear(3),  # Output for 3 classes
    tl.Softmax(dim=-1)
)

net = tl.build(net_arch, torch.randn(1, *X_train[0].shape))

In [ ]:
net

In [ ]:
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping, Checkpoint

# Callbacks.
cp = Checkpoint(dirname='skorch_cp')
es = EarlyStopping(monitor='valid_loss', patience=9)

cnn = NeuralNetClassifier(
    net,
    max_epochs=100,
    batch_size=100,
    lr=0.002,
    optimizer=torch.optim.Adam,
    callbacks=[cp, es],
)

In [ ]:
cnn.fit(X_train, y_train)

In [ ]:
plt.plot(cnn.history[:, 'train_loss'], label='Train loss')
plt.plot(cnn.history[:, 'valid_loss'], label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
cnn.load_params(checkpoint=cp)

y_pred = cnn.predict(X_val)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_pred, y_val))

In [ ]:
y_probs = cnn.predict_proba(X_val)
y_probs[:10]

In [ ]:
import utils

utils.visualize(X_val, y_val, y_probs,
                ncols=5, nrows=3,
                shape=(32, 32),
                classes=encoder.classes_
               )

## The kernels

In [ ]:
w1 = cnn.module_[0].weight.detach().numpy()
w1.shape

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=8, figsize=(12, 6))
for w, ax in zip(w1, axs.ravel()):
    ax.imshow(np.sum(w, axis=0))
    ax.axis('off')

In [ ]:
w2 = cnn.module_[3].weight.detach().numpy()

fig, axs = plt.subplots(nrows=2, ncols=8, figsize=(12, 3))
for w, ax in zip(w2, axs.ravel()):
    ax.imshow(np.sum(w, axis=0))
    ax.axis('off')

## Using grid search with this network

In theory, you can also use `scikit-learn` training flow objects to train robust models such as `GridSearchCV`.

This is potentially cool, but I am not 100% certain that the models are initializing for each model. If you want to use this, I suggest reading the `skorch` docs carefully.

In [ ]:
from sklearn.model_selection import GridSearchCV

cnn = NeuralNetClassifier(
    net,
    max_epochs=10,
    batch_size=100,
    optimizer=torch.optim.Adam,
)

params = {
    'lr': [0.001, 0.003],
}

gs = GridSearchCV(cnn, params, refit=False, cv=3, scoring='accuracy')

gs.fit(X_train, y_train)
print(gs.best_score_, gs.best_params_)

## Model persistence

The easiest way to save a model is to `pickle` the trained model object.

In [ ]:
import pickle

# Saving.
with open('torch_classifier.pkl', 'wb') as f:
    pickle.dump(cnn, f)

# Loading.
with open('torch_classifier.pkl', 'rb') as f:
    cnn = pickle.load(f)

In [ ]:
cnn.initialize()
cnn.predict(X_val)

---

&copy; 2020 Agile Scientific